## **LangChain -  FAISS**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [ ]:
# 1) load PDF
pdf_path = 'Hayat-Qulub-Alama-Majlisi.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [ ]:
print(loader)

In [ ]:
# 2) Split into chunks 
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)

In [ ]:
# 3-A) Create Embeddings - Sentenace Transformer Embeddings
embeddings = HuggingFaceBgeEmbeddings(model_name='all-MiniLM-L12-v2', model_kwargs={'device':'cuda'})
embeddings

In [ ]:
# 4) Store Vector Embeddings
vectordb = FAISS.from_documents(chunks, embeddings)
vectordb.save_local("faiss_all_minilm")

In [ ]:
# 5) Retrieve 
retrieve = vectordb.as_retriever(search_type = 'similarity', search_kwargs={"k":10})
retrieve

In [ ]:
from dotenv import load_dotenv
# ---------- Load env var ----------
load_dotenv()

API_KEYS = os.getenv("GEMINI_API_KEYS", "").split(",")
API_KEYS = [k.strip() for k in API_KEYS if k.strip()]
current_key_index = 0
ACCESS_KEY = os.getenv("APP_ACCESS_KEY")

if not API_KEYS:
    print("❌ No Gemini API keys found. Check .env file.")
    raise Exception("Missing GEMINI_API_KEYS")

In [ ]:
# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
If the answer is not in the documents, say "I don’t know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=system_prompt
)

In [ ]:
# 6) LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=os.getenv('GEMINI_KEY_KEY', '')
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retrieve,
    return_source_documents=False,
    chain_type_kwargs = {"prompt":prompt}
)

In [ ]:
# 7) Ask
query = "Why Adam was named Adam? Give reference to it as well"
res = qa.run(query)
print(res)

In [ ]:
q1 = "Why Adam was named Adam? Explain in details"
res = qa.run(q1)
print(res)

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

## **Pinecone Storage**

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
import os
from dotenv import load_dotenv

In [9]:
load_dotenv()
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(INDEX_NAME)


### **Vectorinze Single PDF**

In [20]:
# ---------- begin of pipelines ----------
# load pdf
pdf_path = 'Hayat-al-Qulub-Vol-1.pdf'
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

ValueError: File path Hayat-al-Qulub-Vol-1.pdf is not a valid file or url

In [ ]:
# split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)

In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)

In [ ]:
# Store in vector database - FAISS
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("Hayat-Qulub-Alama-Majlisi-faiss-index")

### **Vectorize multiple PDFs**

In [16]:
#  Store all 3 PDFs in vector database - Pinecone
pdfs = [
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf'
]

all_chunks = []
for i, pdf in enumerate(pdfs):
    loader = PyMuPDFLoader(pdf)
    docs = loader.load()
    print(f"Loaded {i} documents")
    
    # split text into chunks
    print(f"Splitting {i} documents into chunks...")
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    chunks = splitter.split_documents(docs)
    print(f"splitted {i} documents Successfully! ")
    print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

    all_chunks.extend(chunks)
    print(f"Chunk list filled")
    print(f"Total chunks so far: {len(all_chunks)}")
    


Loaded 0 documents
Splitting 0 documents into chunks...
splitted 0 documents Successfully! 
Loaded 2547 chunks from 0 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf
Chunk list filled
Total chunks so far: 2547
Loaded 1 documents
Splitting 1 documents into chunks...
splitted 1 documents Successfully! 
Loaded 3806 chunks from 1 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf
Chunk list filled
Total chunks so far: 6353
Loaded 2 documents
Splitting 2 documents into chunks...
splitted 2 documents Successfully! 
Loaded 1205 chunks from 2 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf
Chunk list filled
Total chunks so far: 7558


In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)


In [ ]:
# Store in vector database - Pinecone
vector_db = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
pinecone_store = vector_db

In [19]:
pinecone_store = vector_db

### **Vectorize multiple PDFs - functions**

In [ ]:
def process_pdf_into_chunks(pdfs):
    pdfs = list(pdfs)
    all_chunks = []
    for i, pdf in enumerate(pdfs):
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} documents")
        
        # split text into chunks
        print(f"Splitting {i} documents into chunks...")
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"splitted {i} documents Successfully! ")
        print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

        all_chunks.extend(chunks)
        print(f"Chunk list filled")
        print(f"Total chunks so far: {len(all_chunks)}")
    
    return all_chunks

def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
    # create embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model, model_kwargs={"device": "cuda"}
    )
    print(f"Created embeddings using model: {embedding_model}")
    
    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store


### **Retrieve QA**

In [9]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
You have to understand the context deeply and answer accordingly.
If the answer is not in the documents, say "I don't know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())


In [ ]:
# Ask a question
query = "Why Adam was named Adam? Give reference to it as well"
res = qa_chain.invoke(query)

print(res)

In the tapestry of Islamic history, the naming of Adam is a tale woven with profound meaning.

It is narrated through authentic chains that Imam Muhammad al-Baqir and Ja‘far as-Sadiq (peace be upon them) said, "Adam was named ‘Adam’ because he was ‘Adeemul Arz’, that is he was created from the face of the earth (from dust)."

So, Adam's name is intrinsically linked to his origin, a reminder of the humble earth from which he was formed.


In [13]:
query1 = """
Kisne likh hai Hayatal Qulub? kiske bare me hai ye kitab? 
kitni jildhe hai iske aur kya har ek jildh ko short me smjha sakte ho?
"""
res2 = qa_chain.invoke(query1)
print(res2)

Aapka sawaal Hayat-ul-Qulub ke baare mein hai. Chaliye, main aapko iske baare mein tafseel se batata hoon:

**Hayat-ul-Qulub kisne likhi hai?**

Hayat-ul-Qulub Allama Muhammad Baqir Majlisi (Rehmatullah Alaih) ne likhi hai. Allama Majlisi ek mashhoor Islamic scholar aur Muhaddith the.

**Ye kitab kiske baare mein hai?**

Ye kitab Ambiya (Prophets), Aimma (Imams), aur deegar buzurg hastiyon ke ahwaal (life events) aur fazail (virtues) par mabni hai. Isme unke zindagi ke waqiyat, unke akhlaq, aur unke maqamat ko tafseel se bayan kiya gaya hai.

**Is kitab ki kitni jildhein (volumes) hain?**

Hayat-ul-Qulub ki teen jildhein hain. Har jildh mein mukhtalif anbiya aur aimma ke baare mein tafseeli malumaat hain.

**Har jildh ka mukhtasar khulasa (brief summary):**

*   **Jildh 1:** Is jildh mein Ambiya-e-Kiram (Prophets) ke ahwaal bayan kiye gaye hain, jaise Hazrat Adam (A.S.), Hazrat Nuh (A.S.), Hazrat Ibrahim (A.S.), Hazrat Musa (A.S.), aur Hazrat Isa (A.S.). Har Nabi ki zindagi ke aham waq

In [ ]:
# quick check for any embedding wrapper you plan to use
vec = embeddings.embed_query("test")
print("type:", type(vec))
print("len:", len(vec))   # MUST be 1024 to match your Pinecone index

## **Sulaym ibne Qays RAG**

### **RAG Pipeline**

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
from dotenv import load_dotenv
import os

c:\Users\sadiq\OneDrive\Documents\projects\rag-for-Hayatal-Qulub\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [6]:
load_dotenv()

pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ")
index = pc.Index(INDEX_NAME)

In [33]:
def process_pdf_into_chunks(pdfs: list):
    print(f'Count of Documents - {len(pdfs)}')

    all_chunks = []
    for i, pdf in enumerate(pdfs):
        i=+1
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} document successfully")

        # split into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"Splitted {i} documment successfully!")

        all_chunks.extend(chunks)
        print(f"Total chunks so far: {len(all_chunks)}")

    return all_chunks
    
def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
     #create embeddings

    embeddings = HuggingFaceEmbeddings(
        model_name = 'BAAI/bge-large-en-v1.5', model_kwargs={"device": "cuda"} 
        )
    print(f"Created embeddings using model: {embedding_model}")

    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        index_name=INDEX_NAME
    )
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store
    

In [34]:
chunks = process_pdf_into_chunks(pdfs=['kitab_sulaim_ibn_qays_al-hilaali.pdf'])

Count of Documents - 1
Loaded 1 document successfully
Splitted 1 documment successfully!
Total chunks so far: 753


In [35]:
vector_store = store_chunks_in_pinecone(chunks, 'BAAI/bge-large-en-v1.5', INDEX_NAME)

Created embeddings using model: BAAI/bge-large-en-v1.5
Stored 753 chunks in Pinecone index: sulaym-ibne-qays


### **Retrieve QA**

In [38]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from system_prompt import system_prompt_text
load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = system_prompt_text

prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())


In [39]:
# Ask a question
query = "Few people attacked the house of Lady Fatimah (SA) and burned the door. what was the entire incident? Explain in details."
res = qa_chain.invoke(query)
print(res)

The events surrounding the attack on the house of Lady Fatimah (SA) are recounted in detail within the narrations of Kitab Sulaym ibn Qays. These accounts paint a vivid and sorrowful picture of the immediate aftermath of the Prophet's (PBUH) passing and the challenges faced by Imam Ali (AS) and his family.

According to this tradition, after the Prophet's (PBUH) death, some individuals sought to secure allegiance for Abu Bakr. Imam Ali (AS), along with some of his loyal companions, remained in the house of Lady Fatimah (SA), mourning the loss of the Prophet (PBUH) and discussing the rightful succession.

Sulaym ibn Qays narrates that Umar, along with a group of people, went to the house of Imam Ali (AS) seeking to compel him to pledge allegiance to Abu Bakr. When they arrived, Lady Fatimah (SA) was behind the door. Umar demanded that Imam Ali (AS) come out and pledge allegiance, threatening to burn the house down if he refused.

Lady Fatimah (SA), standing behind the door, questioned U

## **LangChain Qdrant**

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
import os 

c:\Users\sadiq\OneDrive\Documents\projects\rag-for-Hayatal-Qulub\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# qdrant setup
qdrant_client = QdrantClient(
    host="xyz-example.eu-central.aws.cloud.qdrant.io",
    api_key=os.getenv("QDRANT_API_KEY"),
)

NameError: name 'QdrantClient' is not defined

In [ ]:
# ---------- begin of pipelines ----------
# load pdf
pdf_path = 'Hayat-Qulub-Alama-Majlisi.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()


In [ ]:
# split text 
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)


In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L12-v2", model_kwargs={"device": "cuda"}
)

In [ ]:
# setup vectordb and store embeddings
vector_db_qdrant = qdrant_client.recreate_collection(
    collection_name="hayat_qulub",
    vectors_config=Qdrant.VectorParams(
         size = 768,
        distance=Qdrant.Distance.COSINE
    ),
)


## **LlamaIndex**

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader 
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
docs = SimpleDirectoryReader('pdf_docs').load_data()
docs


In [ ]:
# splitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

In [ ]:
embeddings = HuggingFaceEmbedding(model_name="all-MiniLM-L12-v2")
embeddings

In [ ]:
storage_context = StorageContext.from_defaults(index_store=embeddings)
storage_context

In [ ]:
vector_store = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
vector_store